In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import glob
import warnings
warnings.filterwarnings("ignore")
import datetime
from datetime import date
import time
import re

import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

baseball_path = r"C:\Users\james\Documents\MLB\Data"
download_path = r"C:\Users\james\Downloads"

In [2]:
todaysdate = date.today()
todaysdate_dash = str(todaysdate)
todaysdate = todaysdate_dash.replace("-", "")
todaysdate

'20221112'

In [7]:
def remove_accents(old):
    new = re.sub(r'[àáâãäå]', 'a', old)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new = re.sub(r'[ñ]', 'n', new)
    return new

In [9]:
def scrape_batters():
    # Set driver
    driver = webdriver.Chrome(executable_path=r'C:\Users\james\OneDrive\Documents\MLB\chromedriver.exe')
    # Choose url
    driver.get(f'https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=steameru&team=0&lg=all&players=0')
    # Select element
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ProjectionBoard1_cmdCSV"]'))))
    
    # Find all Fangraphs downloads
    fangraphs_files = glob.glob(os.path.join(download_path, 'FanGraphs Leaderboard*.csv'))
    fangraphs_files.sort(key=os.path.getmtime)

    # Pick the second to last (this will be the highest number (which is above the unnumbered one)
    latest = fangraphs_files[-1]
    print(latest)

    batters_lb = pd.read_csv(os.path.join(download_path, latest), encoding='iso-8859-1')

    filename = "Batters_FG_" + todaysdate + ".csv"
    batters_lb.to_csv(os.path.join(baseball_path, "FanGraphs", "Batters", filename), encoding='iso-8859-1')
    
scrape_batters()

C:\Users\james\Downloads\FanGraphs Leaderboard (12).csv


In [6]:
# Set driver
driver = webdriver.Chrome(executable_path=r'C:\Users\james\OneDrive\Documents\MLB\chromedriver.exe')
# Choose url
driver.get(f'https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=steameru&team=0&lg=all&players=0')
# Select element
driver.execute_script("arguments[0].click();", WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ProjectionBoard1_cmdCSV"]'))))

time.sleep(15)

In [7]:
# Find all Fangraphs downloads
fangraphs_files = glob.glob(os.path.join(download_path, 'FanGraphs Leaderboard*.csv'))
fangraphs_files.sort(key=os.path.getmtime)
  
# Pick the second to last (this will be the highest number (which is above the unnumbered one)
latest = fangraphs_files[-1]
print(latest)

batters_lb = pd.read_csv(os.path.join(download_path, latest), encoding='iso-8859-1')

filename = "Batters_FG_" + todaysdate + ".csv"
batters_lb.to_csv(os.path.join(baseball_path, "FanGraphs", "Batters", filename), encoding='iso-8859-1')


try:
    driver.close()
except:
    pass
    

batters_lb[['First', 'Last']] = batters_lb['Name'].str.split(" ", n=1, expand=True)

batters_lb['First2'] = batters_lb['First'].str.slice(0,2).str.lower()
batters_lb['Last5'] = batters_lb['Last'].str.slice(0,5).str.lower()

batters_lb['First2'] = batters_lb.apply(lambda x: remove_accents(x['First2']), axis=1)
batters_lb['Last5'] = batters_lb.apply(lambda x: remove_accents(x['Last5']), axis=1)
    
batters_lb

C:\Users\james\Downloads\FanGraphs Leaderboard (10).csv


,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,Fld,-1.2,Off,Def,WAR,playerid,First,Last,First2,Last5
0,Aaron Judge,NYY,160,709,581,180,28,0,63,135,...,5.0,NaN,87.0,0.8,11.6,15640,Aaron,Judge,aa,judge
1,Manny Machado,SDP,151,648,581,172,36,1,32,100,...,6.1,NaN,41.7,6.6,7.4,11493,Manny,Machado,ma,macha
2,Nolan Arenado,STL,149,626,562,164,42,1,30,74,...,12.9,NaN,35.5,13.0,7.3,9777,Nolan,Arenado,no,arena
3,Paul Goldschmidt,STL,153,659,568,180,41,0,35,107,...,-4.0,NaN,62.0,-16.1,7.2,9218,Paul,Goldschmidt,pa,golds
4,Freddie Freeman,LAD,160,712,615,198,46,2,20,117,...,3.2,NaN,50.7,-8.7,6.9,5361,Freddie,Freeman,fr,freem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4190,Jeremy Santana,NaN,0,1,1,0,0,0,0,0,...,0.0,NaN,-0.1,0.0,NaN,sa3018599,Jeremy,Santana,je,santa
4191,Robert Hendrick,NaN,0,1,1,0,0,0,0,0,...,0.0,NaN,-0.1,0.0,NaN,sa3015573,Robert,Hendrick,ro,hendr
4192,Jean Estrada,NaN,0,1,1,0,0,0,0,0,...,0.0,NaN,-0.1,0.0,NaN,sa3016153,Jean,Estrada,je,estra
4193,Ruben Yustiz,NaN,0,1,1,0,0,0,0,0,...,0.0,NaN,-0.1,0.0,NaN,sa3016179,Ruben,Yustiz,ru,yusti


In [8]:
batters_fg = batters_lb.copy()
batters_fg['1B'] = batters_fg['H'] - batters_fg['2B'] - batters_fg['3B'] - batters_fg['HR']

hit_list = ['1B', '2B', '3B', 'HR', 'BB', 'HBP', 'SO']

rate_list = ['OBP', 'SLG', 'wOBA']
for stat in hit_list:
    rate = stat + "_rate"
    rate_list.append(rate)
    batters_fg[rate] = batters_fg[stat] / batters_fg['PA']
    
batters_fg['SBA'] = batters_fg['SB'] + batters_fg['CS']
batters_fg['SBO'] = batters_fg['1B'] + batters_fg['BB'] + batters_fg['HBP']
batters_fg['sba_imp'] = batters_fg['SBA'] / batters_fg['SBO']

# Cap imputed SBA 
batters_fg['sba_imp'] = np.where(batters_fg['sba_imp'] > 0.5, 0.5, batters_fg['sba_imp'])



batters_fg['sbr'] = batters_fg['SB'] / batters_fg['SBA']
    
keep_list = ['playerid', 'First2', 'Last5', 'sba_imp', 'sbr'] + rate_list
batters_fg = batters_fg[keep_list]
batters_fg['sbr'].fillna(0.6, inplace=True) # assume 25th percentile 
batters_fg.columns = batters_fg.columns.str.lower()
batters_fg.rename(columns={'1b_rate': 'b1_rate', '2b_rate': 'b2_rate', '3b_rate': 'b3_rate'}, inplace=True)
batters_fg.dropna(inplace=True)
batters_fg.describe()

,sba_imp,sbr,obp,slg,woba,b1_rate,b2_rate,b3_rate,hr_rate,bb_rate,hbp_rate,so_rate
count,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000
mean,0.078738,0.685471,0.292595,0.352126,0.285617,0.135786,0.037708,0.003084,0.022770,0.077876,0.010874,0.250125
std,0.104158,0.250724,0.063760,0.113061,0.067332,0.047396,0.027587,0.005103,0.017801,0.040054,0.013329,0.087852
min,0.000000,0.000000,0.021000,0.004000,0.019000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.600000,0.262250,0.293000,0.252250,0.111111,0.025197,0.000000,0.008705,0.052807,0.000000,0.189681
50%,0.040000,0.622024,0.298000,0.362500,0.292000,0.137090,0.038297,0.000000,0.021834,0.075094,0.007849,0.243243
75%,0.111111,0.884615,0.326000,0.417750,0.324000,0.162920,0.048830,0.004887,0.033508,0.099551,0.014706,0.302045
max,0.500000,1.000000,0.733000,1.221000,0.832000,0.500000,0.500000,0.058824,0.097403,0.285714,0.125000,0.571429


In [9]:
sba_2b_reg = pickle.load(open('sba_2b_20220901.sav', 'rb'))
batters_fg['sba_2b'] = sba_2b_reg.predict(batters_fg[['sba_imp']])

sba_3b_reg = pickle.load(open('sba_3b_20220901.sav', 'rb'))
batters_fg['sba_3b'] = sba_3b_reg.predict(batters_fg[['sba_imp']])

sb_2b_reg = pickle.load(open('sb_2b_20220901.sav', 'rb'))
batters_fg['sb_2b'] = sb_2b_reg.predict(batters_fg[['sbr']])

sb_3b_reg = pickle.load(open('sb_3b_20220901.sav', 'rb'))
batters_fg['sb_3b'] = sb_3b_reg.predict(batters_fg[['sbr']])
batters_fg.describe()

,sba_imp,sbr,obp,slg,woba,b1_rate,b2_rate,b3_rate,hr_rate,bb_rate,hbp_rate,so_rate,sba_2b,sba_3b,sb_2b,sb_3b
count,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000,666.000000
mean,0.078738,0.685471,0.292595,0.352126,0.285617,0.135786,0.037708,0.003084,0.022770,0.077876,0.010874,0.250125,0.048953,0.013909,0.710348,0.628254
std,0.104158,0.250724,0.063760,0.113061,0.067332,0.047396,0.027587,0.005103,0.017801,0.040054,0.013329,0.087852,0.062690,0.022505,0.168033,0.060942
min,0.000000,0.000000,0.021000,0.004000,0.019000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001563,-0.003103,0.250950,0.461639
25%,0.000000,0.600000,0.262250,0.293000,0.252250,0.111111,0.025197,0.000000,0.008705,0.052807,0.000000,0.189681,0.001563,-0.003103,0.653066,0.607479
50%,0.040000,0.622024,0.298000,0.362500,0.292000,0.137090,0.038297,0.000000,0.021834,0.075094,0.007849,0.243243,0.025638,0.005540,0.667827,0.612832
75%,0.111111,0.884615,0.326000,0.417750,0.324000,0.162920,0.048830,0.004887,0.033508,0.099551,0.014706,0.302045,0.068438,0.020904,0.843814,0.676659
max,0.500000,1.000000,0.733000,1.221000,0.832000,0.500000,0.500000,0.058824,0.097403,0.285714,0.125000,0.571429,0.302500,0.104928,0.921144,0.704705


In [10]:
# Import batters data to get handedness data
batter_df = pd.read_csv(os.path.join(baseball_path, "API", "Inputs", "Batters.csv"), encoding='iso-8859-1')
batter_hands = batter_df[['batter', 'batSide_l', 'key_fangraphs']]
batter_hands['b_L'] = np.where(batter_hands['batSide_l'] == "L", 1, 0)
batter_hands.drop(columns={'batSide_l'}, inplace=True)

batters_fg['playerid'] = pd.to_numeric(batters_fg['playerid'], errors='coerce')
print(batters_fg['playerid'])


# Merge onto existing data
batters_fg = batters_fg.merge(batter_hands, left_on='playerid', right_on='key_fangraphs', how='left')
batters_fg['b_L'].fillna(0, inplace=True)

batters_fg

0       15640
1       11493
2        9777
3        9218
4        5361
        ...  
3186    10071
3187    11680
3188     3269
3189     1744
3190     3142
Name: playerid, Length: 666, dtype: int64


,playerid,first2,last5,sba_imp,sbr,obp,slg,woba,b1_rate,b2_rate,...,bb_rate,hbp_rate,so_rate,sba_2b,sba_3b,sb_2b,sb_3b,batter,key_fangraphs,b_L
0,15640,aa,judge,0.091346,0.842105,0.424,0.684,0.457,0.125529,0.039492,...,0.159379,0.008463,0.251058,0.056542,0.016634,0.815324,0.666327,592450.0,15640.0,0.0
1,11493,ma,macha,0.059524,0.900000,0.365,0.529,0.380,0.158951,0.055556,...,0.098765,0.001543,0.208333,0.037389,0.009758,0.854124,0.680399,592518.0,11493.0,0.0
2,9777,no,arena,0.052980,0.625000,0.357,0.532,0.380,0.145367,0.067093,...,0.084665,0.011182,0.115016,0.033450,0.008344,0.669821,0.613556,571448.0,9777.0,0.0
3,9218,pa,golds,0.037037,1.000000,0.403,0.576,0.419,0.157815,0.062215,...,0.121396,0.007587,0.216995,0.023855,0.004899,0.921144,0.704705,502671.0,9218.0,0.0
4,5361,fr,freem,0.072727,0.812500,0.405,0.503,0.389,0.182584,0.064607,...,0.119382,0.007022,0.144663,0.045336,0.012611,0.795482,0.659131,518692.0,5361.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,10071,jo,villa,0.152174,1.000000,0.260,0.302,0.250,0.140271,0.027149,...,0.067873,0.000000,0.266968,0.093152,0.029776,0.921144,0.704705,542340.0,10071.0,0.0
662,11680,el,diaz,0.012658,0.000000,0.281,0.368,0.284,0.135065,0.046753,...,0.064935,0.005195,0.215584,0.009182,-0.000368,0.250950,0.461639,553869.0,11680.0,0.0
663,3269,ro,cano,0.000000,0.600000,0.184,0.192,0.170,0.123810,0.009524,...,0.038095,0.000000,0.238095,0.001563,-0.003103,0.653066,0.607479,429664.0,3269.0,1.0
664,1744,mi,cabre,0.008333,1.000000,0.305,0.318,0.275,0.200000,0.022727,...,0.065909,0.006818,0.234091,0.006579,-0.001302,0.921144,0.704705,408234.0,1744.0,0.0


In [11]:
simple_list = ['so', 'b1', 'b2', 'b3', 'hr', 'bb', 'hbp', 'lo', 'po', 'go', 'fo']

# Read in models
woba_reg = pickle.load(open('woba_20220908.sav', 'rb'))
obp_reg = pickle.load(open('obp_20220908.sav', 'rb'))
slg_reg = pickle.load(open('slg_20220908.sav', 'rb'))

fg_vs_lhp = pickle.load(open('fg_vs_lhp_20220905.sav', 'rb'))
fg_vs_rhp = pickle.load(open('fg_vs_rhp_20220905.sav', 'rb'))
fg_vs_lhb = pickle.load(open('fg_vs_lhb_20220905.sav', 'rb'))
fg_vs_rhb = pickle.load(open('fg_vs_rhb_20220905.sav', 'rb'))

In [12]:
# Create vs lhp dataframe
vs_lhp_preds = fg_vs_lhp.predict_proba(batters_fg[['b_L', 'b1_rate', 'b2_rate', 'b3_rate', 'hr_rate', 'bb_rate', 'hbp_rate', 'so_rate']])
vs_lhp_df = pd.DataFrame(vs_lhp_preds, columns=fg_vs_lhp.classes_)

vs_lhp_df['woba'] = woba_reg.predict(vs_lhp_df[simple_list])
vs_lhp_df['obp'] = obp_reg.predict(vs_lhp_df[simple_list])
vs_lhp_df['slg'] = slg_reg.predict(vs_lhp_df[simple_list])

vs_lhp_df = vs_lhp_df.add_suffix("_l")

vs_lhp_df

,b1_l,b2_l,b3_l,bb_l,fo_l,go_l,hbp_l,hr_l,lo_l,po_l,so_l,woba_l,obp_l,slg_l
0,0.119257,0.030137,0.001023,0.175879,0.115585,0.160929,0.007669,0.071539,0.032773,0.038281,0.246928,0.420185,0.405504,0.472549
1,0.152308,0.065670,0.003441,0.108497,0.124228,0.206814,0.004322,0.047269,0.048661,0.039600,0.199190,0.399002,0.381506,0.485649
2,0.152574,0.082191,0.002986,0.093677,0.160585,0.191851,0.006196,0.045327,0.073121,0.083894,0.107598,0.406466,0.382951,0.510696
3,0.148322,0.062766,0.003076,0.133498,0.116508,0.193438,0.005541,0.049650,0.043656,0.035956,0.207589,0.414286,0.402854,0.484734
4,0.169672,0.054821,0.006408,0.092489,0.114123,0.262609,0.009381,0.017996,0.056038,0.034574,0.181891,0.337434,0.350766,0.374023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0.145732,0.037285,0.005386,0.062156,0.118006,0.277336,0.002356,0.011080,0.061051,0.036440,0.243172,0.252171,0.263995,0.280853
662,0.125227,0.066394,0.004988,0.064805,0.136541,0.253812,0.005104,0.023007,0.064340,0.052224,0.203559,0.298620,0.289524,0.366852
663,0.118103,0.012799,0.000326,0.027805,0.098667,0.416386,0.001601,0.003451,0.055246,0.062520,0.203097,0.148679,0.164084,0.157242
664,0.197043,0.028129,0.001377,0.063529,0.106434,0.291180,0.002848,0.011047,0.060380,0.022287,0.215746,0.280840,0.303971,0.302063


In [13]:
# Create vs rhp dataframe
vs_rhp_preds = fg_vs_rhp.predict_proba(batters_fg[['b_L', 'b1_rate', 'b2_rate', 'b3_rate', 'hr_rate', 'bb_rate', 'hbp_rate', 'so_rate']])
vs_rhp_df = pd.DataFrame(vs_rhp_preds, columns=fg_vs_rhp.classes_)

vs_rhp_df['woba'] = woba_reg.predict(vs_rhp_df[simple_list])
vs_rhp_df['obp'] = obp_reg.predict(vs_rhp_df[simple_list])
vs_rhp_df['slg'] = slg_reg.predict(vs_rhp_df[simple_list])

vs_rhp_df = vs_rhp_df.add_suffix("_r")

vs_rhp_df

,b1_r,b2_r,b3_r,bb_r,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r
0,0.119596,0.046951,0.000739,0.142267,0.111419,0.164581,0.006572,0.085608,0.040693,0.035751,0.245824,0.446389,0.401733,0.562378
1,0.158244,0.059423,0.001418,0.090272,0.126635,0.210287,0.002388,0.048489,0.053352,0.039676,0.209816,0.381689,0.360234,0.477119
2,0.143764,0.070622,0.002763,0.073931,0.177603,0.215355,0.010436,0.048937,0.064881,0.068701,0.123008,0.380615,0.350453,0.494011
3,0.155219,0.069906,0.001129,0.110160,0.115160,0.193847,0.005800,0.054612,0.049955,0.033542,0.210670,0.420636,0.396826,0.520363
4,0.196651,0.067860,0.002176,0.118402,0.124376,0.239778,0.005064,0.031276,0.055138,0.026636,0.132644,0.413249,0.421429,0.466652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0.140727,0.031047,0.005582,0.056217,0.120470,0.249957,0.001966,0.011262,0.064952,0.040857,0.276963,0.236181,0.246801,0.264412
662,0.138593,0.045832,0.003625,0.053480,0.141358,0.248438,0.003994,0.020481,0.067029,0.056411,0.220758,0.268477,0.266006,0.324127
663,0.119591,0.019512,0.001021,0.034613,0.136324,0.308115,0.001819,0.009546,0.074085,0.057121,0.238253,0.177052,0.186102,0.199043
664,0.205006,0.028519,0.001356,0.056286,0.109006,0.259566,0.006168,0.009429,0.060988,0.027729,0.235946,0.282395,0.306764,0.305573


In [14]:
batters_fg = batters_fg[['playerid', 'batter', 'b_L', 'first2', 'last5', 'sba_2b', 'sba_3b', 'sb_2b', 'sb_3b']]
batters_fg.rename(columns={'first2': 'First2', 'last5':'Last5'},inplace=True)
batters_fg = pd.concat([batters_fg, vs_lhp_df, vs_rhp_df], axis=1)
batters_fg


,playerid,batter,b_L,First2,Last5,sba_2b,sba_3b,sb_2b,sb_3b,b1_l,...,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r
0,15640,592450.0,0.0,aa,judge,0.056542,0.016634,0.815324,0.666327,0.119257,...,0.111419,0.164581,0.006572,0.085608,0.040693,0.035751,0.245824,0.446389,0.401733,0.562378
1,11493,592518.0,0.0,ma,macha,0.037389,0.009758,0.854124,0.680399,0.152308,...,0.126635,0.210287,0.002388,0.048489,0.053352,0.039676,0.209816,0.381689,0.360234,0.477119
2,9777,571448.0,0.0,no,arena,0.033450,0.008344,0.669821,0.613556,0.152574,...,0.177603,0.215355,0.010436,0.048937,0.064881,0.068701,0.123008,0.380615,0.350453,0.494011
3,9218,502671.0,0.0,pa,golds,0.023855,0.004899,0.921144,0.704705,0.148322,...,0.115160,0.193847,0.005800,0.054612,0.049955,0.033542,0.210670,0.420636,0.396826,0.520363
4,5361,518692.0,1.0,fr,freem,0.045336,0.012611,0.795482,0.659131,0.169672,...,0.124376,0.239778,0.005064,0.031276,0.055138,0.026636,0.132644,0.413249,0.421429,0.466652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,10071,542340.0,0.0,jo,villa,0.093152,0.029776,0.921144,0.704705,0.145732,...,0.120470,0.249957,0.001966,0.011262,0.064952,0.040857,0.276963,0.236181,0.246801,0.264412
662,11680,553869.0,0.0,el,diaz,0.009182,-0.000368,0.250950,0.461639,0.125227,...,0.141358,0.248438,0.003994,0.020481,0.067029,0.056411,0.220758,0.268477,0.266006,0.324127
663,3269,429664.0,1.0,ro,cano,0.001563,-0.003103,0.653066,0.607479,0.118103,...,0.136324,0.308115,0.001819,0.009546,0.074085,0.057121,0.238253,0.177052,0.186102,0.199043
664,1744,408234.0,0.0,mi,cabre,0.006579,-0.001302,0.921144,0.704705,0.197043,...,0.109006,0.259566,0.006168,0.009429,0.060988,0.027729,0.235946,0.282395,0.306764,0.305573


# Pitchers

In [15]:
# Set driver
driver = webdriver.Chrome(executable_path=r'C:\Users\james\OneDrive\Documents\MLB\chromedriver.exe')
# Choose url
driver.get(f'https://www.fangraphs.com/projections.aspx?pos=all&stats=pit&type=steameru&team=0&lg=all&players=0')
# Select element
driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ProjectionBoard1_cmdCSV"]'))))

time.sleep(15)

In [16]:
# Find all Fangraphs downloads
fangraphs_files = glob.glob(os.path.join(download_path, 'FanGraphs Leaderboard*.csv'))
fangraphs_files.sort(key=os.path.getmtime)
  
# Pick the second to last (this will be the highest number (which is above the unnumbered one)
latest = fangraphs_files[-1]
print(latest)

pitchers_lb = pd.read_csv(os.path.join(download_path, latest), encoding='iso-8859-1')

filename = "Pitchers_FG_" + todaysdate + ".csv"
pitchers_lb.to_csv(os.path.join(baseball_path, "FanGraphs", "Pitchers", filename), encoding='iso-8859-1')

try:
    driver.close()
except:
    pass
    

pitchers_lb[['First', 'Last']] = pitchers_lb['Name'].str.split(" ", n=1, expand=True)

pitchers_lb['First2'] = pitchers_lb['First'].str.slice(0,2).str.lower()
pitchers_lb['Last5'] = pitchers_lb['Last'].str.slice(0,5).str.lower()

pitchers_lb['First2'] = pitchers_lb.apply(lambda x: remove_accents(x['First2']), axis=1)
pitchers_lb['Last5'] = pitchers_lb.apply(lambda x: remove_accents(x['Last5']), axis=1)
    
pitchers_lb

C:\Users\james\Downloads\FanGraphs Leaderboard (11).csv


,Name,Team,GS,G,IP,W,L,QS,SV,HLD,...,ERA,FIP,-1,WAR,RA9-WAR,playerid,First,Last,First2,Last5
0,Aaron Nola,PHI,32,32,206.0,NaN,NaN,18,0,0,...,3.25,2.58,NaN,6.3,5.5,16149,Aaron,Nola,aa,nola
1,Carlos Rodón,SFG,31,31,179.0,NaN,NaN,16,0,0,...,2.88,2.25,NaN,6.2,5.3,16137,Carlos,Rodón,ca,rodon
2,Justin Verlander,HOU,28,28,176.0,NaN,NaN,21,0,0,...,1.76,2.49,NaN,6.1,7.1,8700,Justin,Verlander,ju,verla
3,Kevin Gausman,TOR,31,31,177.0,NaN,NaN,17,0,0,...,3.35,2.39,NaN,5.7,3.4,14107,Kevin,Gausman,ke,gausm
4,Sandy Alcantara,MIA,32,32,230.0,NaN,NaN,24,0,0,...,2.29,2.99,NaN,5.7,7.8,18684,Sandy,Alcantara,sa,alcan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,Matt Swarmer,CHC,5,11,35.0,NaN,NaN,2,0,0,...,5.04,7.29,NaN,-0.8,-0.6,19705,Matt,Swarmer,ma,swarm
5976,Caleb Smith,ARI,1,43,69.0,NaN,NaN,0,0,4,...,4.22,5.66,NaN,-0.9,0.1,14875,Caleb,Smith,ca,smith
5977,Elvin Rodriguez,DET,5,7,31.0,NaN,NaN,0,0,0,...,10.41,8.18,NaN,-0.9,-1.5,20962,Elvin,Rodriguez,el,rodri
5978,Elieser Hernandez,MIA,9,19,59.0,NaN,NaN,1,0,0,...,6.28,6.68,NaN,-1.0,-1.2,16933,Elieser,Hernandez,el,herna


In [17]:
pitchers_fg = pitchers_lb.copy()

# Import batters data to get handedness data
pitcher_df = pd.read_csv(os.path.join(baseball_path, "API", "Inputs", "Pitchers.csv"), encoding='iso-8859-1')
pitcher_hands = pitcher_df[['pitcher', 'pitchHand_l', 'key_fangraphs']]
pitcher_hands['p_L'] = np.where(pitcher_hands['pitchHand_l'] == "L", 1, 0)
pitcher_hands.drop(columns={'pitchHand_l'}, inplace=True)


pitchers_fg['playerid'] = pd.to_numeric(pitchers_fg['playerid'], errors='coerce')

# Merge onto existing data
pitchers_fg = pitchers_fg.merge(pitcher_hands, left_on='playerid', right_on='key_fangraphs', how='left')
pitchers_fg['p_L'].fillna(0, inplace=True)


pitchers_fg['H/9'] = pitchers_fg['H'] / pitchers_fg['IP'] * 9
pitchers_fg['HR/9'] = pitchers_fg['HR'] / pitchers_fg['IP'] * 9
    
keep_list = ['playerid', 'pitcher', 'First2', 'Last5', 'p_L', 'H/9', 'HR/9', 'K/9', 'BB/9'] 
pitchers_fg = pitchers_fg[keep_list]
pitchers_fg


,playerid,pitcher,First2,Last5,p_L,H/9,HR/9,K/9,BB/9
0,16149.0,605400.0,aa,nola,0.0,7.383495,0.830097,10.32,1.28
1,16137.0,607074.0,ca,rodon,1.0,6.636872,0.603352,11.98,2.63
2,8700.0,434378.0,ju,verla,0.0,5.982955,0.613636,9.52,1.49
3,14107.0,592332.0,ke,gausm,0.0,9.661017,0.762712,10.55,1.45
4,18684.0,645261.0,sa,alcan,0.0,6.847826,0.626087,8.15,1.97
...,...,...,...,...,...,...,...,...,...
668275,19705.0,NaN,ma,swarm,0.0,8.742857,3.085714,9.49,5.25
668276,14875.0,592761.0,ca,smith,1.0,7.434783,1.826087,8.19,4.99
668277,20962.0,NaN,el,rodri,0.0,12.483871,3.483871,7.58,4.51
668278,16933.0,622694.0,el,herna,0.0,9.762712,2.898305,8.79,3.38


In [18]:
# Create vs lhb dataframe
vs_lhb_preds = fg_vs_lhb.predict_proba(pitchers_fg[['p_L', 'H/9', 'HR/9', 'K/9', 'BB/9']])
vs_lhb_df = pd.DataFrame(vs_lhb_preds, columns=fg_vs_lhb.classes_)

vs_lhb_df['woba'] = woba_reg.predict(vs_lhb_df[simple_list])
vs_lhb_df['obp'] = obp_reg.predict(vs_lhb_df[simple_list])
vs_lhb_df['slg'] = slg_reg.predict(vs_lhb_df[simple_list])

vs_lhb_df = vs_lhb_df.add_suffix("_l")

vs_lhb_df

,b1_l,b2_l,b3_l,bb_l,fo_l,go_l,hbp_l,hr_l,lo_l,po_l,so_l,woba_l,obp_l,slg_l
0,0.135107,0.035757,0.009788,0.039698,0.128895,0.191879,0.011056,0.018503,0.059138,0.049909,0.320270,0.254030,0.249909,0.313698
1,0.118334,0.022144,0.006616,0.067923,0.090993,0.204821,0.014457,0.011510,0.042786,0.036895,0.383521,0.224380,0.240985,0.232796
2,0.117064,0.035385,0.007951,0.052701,0.134748,0.194984,0.011153,0.020433,0.065290,0.056877,0.303412,0.247689,0.244687,0.296982
3,0.164579,0.041345,0.006276,0.034856,0.116443,0.242214,0.010270,0.012128,0.041133,0.034377,0.296379,0.264439,0.269454,0.318031
4,0.133277,0.041428,0.005731,0.067052,0.127817,0.257807,0.010843,0.020215,0.065561,0.040822,0.229447,0.275336,0.278546,0.317810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668275,0.108402,0.062522,0.006784,0.163725,0.175576,0.125903,0.006395,0.072166,0.033623,0.040776,0.204128,0.452665,0.419994,0.546364
668276,0.109942,0.044001,0.007630,0.130696,0.141433,0.165320,0.015631,0.038758,0.045821,0.058020,0.242749,0.346722,0.346657,0.381805
668277,0.156198,0.047359,0.004283,0.153045,0.130682,0.200313,0.005488,0.083642,0.048829,0.028556,0.141605,0.487557,0.450015,0.602455
668278,0.109188,0.060601,0.007509,0.117403,0.171820,0.165419,0.008460,0.090723,0.043704,0.038427,0.186747,0.460036,0.393884,0.621246


In [19]:
# Create vs rhb dataframe
vs_rhb_preds = fg_vs_rhb.predict_proba(pitchers_fg[['p_L', 'H/9', 'HR/9', 'K/9', 'BB/9']])
vs_rhb_df = pd.DataFrame(vs_rhb_preds, columns=fg_vs_rhb.classes_)

vs_rhb_df['woba'] = woba_reg.predict(vs_rhb_df[simple_list])
vs_rhb_df['obp'] = obp_reg.predict(vs_rhb_df[simple_list])
vs_rhb_df['slg'] = slg_reg.predict(vs_rhb_df[simple_list])

vs_rhb_df = vs_rhb_df.add_suffix("_r")

vs_rhb_df

,b1_r,b2_r,b3_r,bb_r,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r
0,0.142251,0.031817,0.002009,0.039765,0.137870,0.188915,0.009398,0.021795,0.062198,0.055248,0.308735,0.248589,0.247034,0.302048
1,0.115884,0.039664,0.000106,0.055830,0.054393,0.270717,0.004080,0.013092,0.033256,0.017651,0.395329,0.221347,0.228655,0.248421
2,0.127895,0.028309,0.001645,0.045365,0.161127,0.174888,0.008660,0.018402,0.064429,0.061480,0.307800,0.227184,0.230276,0.265388
3,0.201155,0.035502,0.001553,0.026338,0.077543,0.277938,0.012027,0.014911,0.039395,0.037002,0.276634,0.283025,0.291488,0.340790
4,0.135550,0.033161,0.000832,0.048157,0.093449,0.331541,0.009721,0.013097,0.052272,0.034244,0.247976,0.230492,0.240519,0.259483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668275,0.084352,0.020966,0.000224,0.095546,0.174085,0.119822,0.071596,0.056541,0.064473,0.042005,0.270391,0.336055,0.329224,0.380371
668276,0.098099,0.038941,0.001131,0.144715,0.156137,0.130736,0.010132,0.043758,0.066979,0.072616,0.236758,0.335509,0.336776,0.358055
668277,0.151496,0.047069,0.003674,0.065024,0.096027,0.248313,0.016962,0.065669,0.047446,0.048108,0.210212,0.392388,0.349894,0.527222
668278,0.134489,0.049022,0.004233,0.062927,0.122962,0.155551,0.020133,0.051836,0.067554,0.061592,0.269700,0.352902,0.322640,0.461037


In [20]:
pitchers_fg = pitchers_fg[['playerid', 'pitcher', 'p_L', 'First2', 'Last5']]

pitchers_fg = pd.concat([pitchers_fg, vs_lhb_df, vs_rhb_df], axis=1)
pitchers_fg


,playerid,pitcher,p_L,First2,Last5,b1_l,b2_l,b3_l,bb_l,fo_l,...,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r
0,16149.0,605400.0,0.0,aa,nola,0.135107,0.035757,0.009788,0.039698,0.128895,...,0.137870,0.188915,0.009398,0.021795,0.062198,0.055248,0.308735,0.248589,0.247034,0.302048
1,16137.0,607074.0,1.0,ca,rodon,0.118334,0.022144,0.006616,0.067923,0.090993,...,0.054393,0.270717,0.004080,0.013092,0.033256,0.017651,0.395329,0.221347,0.228655,0.248421
2,8700.0,434378.0,0.0,ju,verla,0.117064,0.035385,0.007951,0.052701,0.134748,...,0.161127,0.174888,0.008660,0.018402,0.064429,0.061480,0.307800,0.227184,0.230276,0.265388
3,14107.0,592332.0,0.0,ke,gausm,0.164579,0.041345,0.006276,0.034856,0.116443,...,0.077543,0.277938,0.012027,0.014911,0.039395,0.037002,0.276634,0.283025,0.291488,0.340790
4,18684.0,645261.0,0.0,sa,alcan,0.133277,0.041428,0.005731,0.067052,0.127817,...,0.093449,0.331541,0.009721,0.013097,0.052272,0.034244,0.247976,0.230492,0.240519,0.259483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668275,19705.0,NaN,0.0,ma,swarm,0.108402,0.062522,0.006784,0.163725,0.175576,...,0.174085,0.119822,0.071596,0.056541,0.064473,0.042005,0.270391,0.336055,0.329224,0.380371
668276,14875.0,592761.0,1.0,ca,smith,0.109942,0.044001,0.007630,0.130696,0.141433,...,0.156137,0.130736,0.010132,0.043758,0.066979,0.072616,0.236758,0.335509,0.336776,0.358055
668277,20962.0,NaN,0.0,el,rodri,0.156198,0.047359,0.004283,0.153045,0.130682,...,0.096027,0.248313,0.016962,0.065669,0.047446,0.048108,0.210212,0.392388,0.349894,0.527222
668278,16933.0,622694.0,0.0,el,herna,0.109188,0.060601,0.007509,0.117403,0.171820,...,0.122962,0.155551,0.020133,0.051836,0.067554,0.061592,0.269700,0.352902,0.322640,0.461037


# Export 

In [21]:
# Batters
batters_fg.to_csv(os.path.join(baseball_path, "FanGraphs", "Batters_FG.csv"), encoding='iso-8859-1')
# Pitchers
pitchers_fg.to_csv(os.path.join(baseball_path, "FanGraphs", "Pitchers_FG.csv"), encoding='iso-8859-1')


batters_fg.query('playerid == 671739')

,playerid,batter,b_L,First2,Last5,sba_2b,sba_3b,sb_2b,sb_3b,b1_l,...,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r


In [22]:
batters_fg.dropna(subset='playerid', axis=0, inplace=True)
pitchers_fg.dropna(subset='playerid', axis=0, inplace=True)

batters_fg.query('Last5 == "harri"')

,playerid,batter,b_L,First2,Last5,sba_2b,sba_3b,sb_2b,sb_3b,b1_l,...,fo_r,go_r,hbp_r,hr_r,lo_r,po_r,so_r,woba_r,obp_r,slg_r
25,25931,NaN,0.0,mi,harri,0.131379,0.043499,0.860217,0.682609,0.168378,...,0.122687,0.214717,0.011551,0.036723,0.056923,0.040735,0.241237,0.342330,0.323700,0.442558
175,8202,543281.0,0.0,jo,harri,0.018438,0.002955,0.697746,0.623683,0.159103,...,0.131210,0.264502,0.030414,0.014594,0.069342,0.048262,0.171348,0.302043,0.315336,0.346306
360,17216,656509.0,0.0,mo,harri,0.152031,0.050913,0.921144,0.704705,0.059601,...,0.055259,0.123879,0.038643,0.026801,0.018641,0.026422,0.506759,0.249776,0.269040,0.236343


# Merge

Batters

In [23]:
# Merged on MLB id
batter_merged1 = batter_df.merge(batters_fg, on='batter', how='inner', suffixes=("", "_fg1"))
# Merged on name
batter_merged2 = batter_df.merge(batters_fg, on=['First2', 'Last5'], how='inner', suffixes=("", "_fg2"))

# List of batters who merged
batter_merged1_list = batter_merged1['batterName'].unique()
batter_merged2_list = batter_merged2['batterName'].unique()

# print(batter_merged1.query('batterName == "Michael Harris II"'))
# print(batters_fg.query('batter == 671739'))

# Keep only those in the second group who didn't merge in the first group
batter_merged2 = batter_merged2[~batter_merged2['batterName'].isin(batter_merged1_list)]

# Append them together
batter_merged = pd.concat([batter_merged1, batter_merged2], axis=0)

# Replace with FanGraphs imputations if sample is small 
stat_list = ['so', 'b1', 'b2', 'b3', 'hr', 'bb', 'hbp', 'lo', 'po', 'go', 'fo', 'woba', 'obp', 'slg']

# Fill in missings with 0 so the < will work 
batter_merged['pa_b_l'].fillna(0, inplace=True)
batter_merged['pa_b_r'].fillna(0, inplace=True)

# Replace small sample size stats with FanGraphs imputations
# Left
for stat in stat_list:
    short = stat + "_b_l"
    long = stat + "_b_long_l"
    fangraphs = stat + "_l"
    batter_merged[short] = np.where(batter_merged['pa_b_l'] < 40, batter_merged[fangraphs], batter_merged[short])
    batter_merged[long] = np.where(batter_merged['pa_b_l'] < 40, batter_merged[fangraphs], batter_merged[long])

# Right
for stat in stat_list:
    short = stat + "_b_r"
    long = stat + "_b_long_r"
    fangraphs = stat + "_r"
    batter_merged[short] = np.where(batter_merged['pa_b_r'] < 40, batter_merged[fangraphs], batter_merged[short])
    batter_merged[long] = np.where(batter_merged['pa_b_r'] < 40, batter_merged[fangraphs], batter_merged[long])
    
batter_merged['imp'] = np.where(batter_merged['pa_b_r'] < 40, 1, 0)
    
batter_merged.to_csv(os.path.join(baseball_path, "Model Input", "Batters.csv"), encoding='iso-8859-1')

Pitchers

In [24]:
# Merged on MLB id
pitcher_merged1 = pitcher_df.merge(pitchers_fg, on='pitcher', how='inner', suffixes=("", "_fg1"))
# Merged on name
pitcher_merged2 = pitcher_df.merge(pitchers_fg, on=['First2', 'Last5'], how='inner', suffixes=("", "_fg2"))

# List of batters who merged
pitcher_merged1_list = pitcher_merged1['pitcherName'].unique()
pitcher_merged2_list = pitcher_merged2['pitcherName'].unique()

# Keep only those in the second group who didn't merge in the first group
pitcher_merged2 = pitcher_merged2[~pitcher_merged2['pitcherName'].isin(pitcher_merged1_list)]

# Append them together
pitcher_merged = pd.concat([pitcher_merged1, pitcher_merged2], axis=0)

# Replace with FanGraphs imputations if sample is small 
stat_list = ['so', 'b1', 'b2', 'b3', 'hr', 'bb', 'hbp', 'lo', 'po', 'go', 'fo', 'woba', 'obp', 'slg']

# Fill in missings with 0 so the < will work 
pitcher_merged['pa_p_l'].fillna(0, inplace=True)
pitcher_merged['pa_p_r'].fillna(0, inplace=True)

# Replace small sample size stats with FanGraphs imputations
# Left
for stat in stat_list:
    short = stat + "_p_l"
    long = stat + "_p_long_l"
    fangraphs = stat + "_l"
    pitcher_merged[short] = np.where(pitcher_merged['pa_p_l'] < 40, pitcher_merged[fangraphs], pitcher_merged[short])
    pitcher_merged[long] = np.where(pitcher_merged['pa_p_l'] < 40, pitcher_merged[fangraphs], pitcher_merged[long])

# Right
for stat in stat_list:
    short = stat + "_p_r"
    long = stat + "_p_long_r"
    fangraphs = stat + "_r"
    pitcher_merged[short] = np.where(pitcher_merged['pa_p_r'] < 40, pitcher_merged[fangraphs], pitcher_merged[short])
    pitcher_merged[long] = np.where(pitcher_merged['pa_p_r'] < 40, pitcher_merged[fangraphs], pitcher_merged[long])
    

pitcher_merged['imp'] = np.where(pitcher_merged['pa_p_r'] < 40, 1, 0)
    
pitcher_merged.to_csv(os.path.join(baseball_path, "Model Input", "Pitchers.csv"), encoding='iso-8859-1')

In [25]:
# Problem: all rookies are righties. This is because they don't have a fangraphs ID in chadwick
# Can add manually, or can do some sort of merge with fg splits but that's really messy
# I'd try to do a nice fix of chadwick somehow. Merging it with rosters somehow idk

In [26]:
print("Code was last run on: " + str(datetime.date.today()))

Code was last run on: 2022-10-14
